<a href="https://colab.research.google.com/github/swetha-rrr/LLM/blob/main/lab_8_2348567.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install datasets transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import pandas as pd


In [5]:
# Step 2: Load the dataset
# Using the IMDB movie reviews dataset for binary sentiment classification (positive/negative)
dataset = load_dataset("imdb")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
# Step 3: Load the tokenizer
model_name_or_path = "bert-base-uncased"  # Pretrained BERT model
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
# Step 4: Tokenize the dataset
def preprocess_data(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=128)



In [8]:
# Apply the tokenizer to the dataset
encoded_dataset = dataset.map(preprocess_data, batched=True)



Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
# Step 5: Prepare the data for PyTorch
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])



In [10]:
# Step 6: Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, num_labels=2)



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Step 7: Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# Step 8: Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer,
)



In [ ]:
# Step 9: Train the model
trainer.train()


Epoch,Training Loss,Validation Loss


In [ ]:

# Step 10: Evaluate the model
eval_results = trainer.evaluate()

print(f"Evaluation results: {eval_results}")



In [ ]:
# Step 11: Save the fine-tuned model
model.save_pretrained("./fine_tuned_sentiment_model")
tokenizer.save_pretrained("./fine_tuned_sentiment_model")



In [ ]:
# Step 12: Make predictions with the fine-tuned model
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted_class = predictions.argmax().item()
    return "Positive" if predicted_class == 1 else "Negative"

# Example prediction
sample_text = "I really enjoyed this movie!"
predicted_sentiment = predict_sentiment(sample_text)
print(f"Predicted sentiment: {predicted_sentiment}")

Develop a Python script for handling multimodal inputs.

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, ViTFeatureExtractor, ViTModel
from torch import nn
import torch
from PIL import Image
import requests
from io import BytesIO



In [ ]:

text_model_name = "bert-base-uncased"
image_model_name = "google/vit-base-patch16-224"



In [ ]:
# Load tokenizer for the text model
text_tokenizer = AutoTokenizer.from_pretrained(text_model_name)
# Load feature extractor for the image model
image_feature_extractor = ViTFeatureExtractor.from_pretrained(image_model_name)



In [ ]:
# Load the models
text_model = AutoModel.from_pretrained(text_model_name)
image_model = ViTModel.from_pretrained(image_model_name)

In [ ]:
# Step 3: Define a multimodal model by combining the text and image models
class MultimodalModel(nn.Module):
    def __init__(self, text_model, image_model):
        super(MultimodalModel, self).__init__()
        self.text_model = text_model
        self.image_model = image_model
        self.fc = nn.Linear(text_model.config.hidden_size + image_model.config.hidden_size, 2)  # Binary classification

    def forward(self, text_inputs, image_inputs):
        text_outputs = self.text_model(**text_inputs).last_hidden_state[:, 0, :]
        image_outputs = self.image_model(**image_inputs).last_hidden_state[:, 0, :]
        combined = torch.cat((text_outputs, image_outputs), dim=1)
        logits = self.fc(combined)
        return logits



In [ ]:
# Initialize the multimodal model
model = MultimodalModel(text_model, image_model)



In [ ]:
# Step 4: Preprocess the text and image inputs
def preprocess_text(text):
    return text_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

def preprocess_image(image_url):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    return image_feature_extractor(images=image, return_tensors="pt")




In [ ]:
# Example inputs
text = "A beautiful sunset over the mountains."
image_url = "https://example.com/sunset.jpg"



In [ ]:
# Preprocess the inputs
text_inputs = preprocess_text(text)
image_inputs = preprocess_image(image_url)



In [ ]:
# Step 5: Make a prediction
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    logits = model(text_inputs, image_inputs)



In [ ]:
# Step 6: Interpret the results
predictions = torch.softmax(logits, dim=1)
predicted_class = predictions.argmax(dim=1).item()

# Output the predicted class
print(f"Predicted class: {predicted_class}")